In [2]:
import numpy as np
import sys,os
sys.path.append(os.pardir)
from util import im2col

x1 = np.random.rand(1,3,7,7)
coll = im2col(x1,5,5,stride=1,pad=0)
print(coll.shape)

x2 = np.random.rand(10,3,7,7)
coll = im2col(x2,5,5,stride=1,pad=0)
print(coll.shape)

(9, 75)
(90, 75)


In [4]:
class Convolution:
    def __init__(self,w,b,stride=1,pad=0):
        self.w = w
        self.b = b
        self.stride = stride
        self.pad = pad
    
    def forward(self,x):
        fn,c,fh,fw = self.w.shape
        n,c,h,w = x.shape
        out_h = int(1 + (h + 2*self.pad - fh) / self.stride)
        out_w = int(1 + (w + 2*self.pad - fh) / self.stride)
        
        col = im2col(x,fh,fw,self.stride,self.pad)
        col_w = self.w.reshape(fn, -1).T
        out = np.dot(col,col_w) + self.b
        
        out = out.reshape(n, out_h,out_w, -1).transpose(0,3,1,2)
        
        return out

In [5]:
class Pooling:
    def __init__(self,pool_h,pool_w,stride=1,pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
    
    def forward(self,x):
        n,c,h,w = x.shape
        out_h = int(1 + (h - self.pool_h) / self.stride)
        out_w = int(1 + (w - self.pool_w) / self.stride)
        
        col = im2col(x,self.pool_h,self.pool_w,self.stride,self.pad)
        col = col.reshape(-1,self.pool_h * self.pool_w)
        
        out = np.max(col,axis=1)
        out = out.reshape(n,out_h,out_w,c).transpose(0,3,1,2)
        
        return out

In [9]:
class SimpleConvNet:
    def __init__(self,input_dim=(1,28,28),conv_param={'filter_num':30,'filter_size':5,'pad':0,'stride':1},
                 hidden_size=100, output_size=10,weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size =conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2 * filter_pad)/\
                            filter_stride + 1
        pool_output_size = int(filter_num*(conv_output_size/2) * (conv_output_size/2))
        
        self.params = {}
        self.params['w1'] = weight_init_std * \
                            np.random.randn(filter_num,input_dim[0],filter_size,filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['w2'] = weight_init_std * \
                            np.random.randn(pool_output_size,hidden_size)
        self.params['b2'] = np.zeros(filter_num)
        self.params['w2'] = weight_init_std * \
                            np.random.randn(hidden_size,output_size)
        self.params['b3'] = np.zeros(filter_num)
        
        self.layers = OrderedDict()
        self.layers['conv1'] = Convolution(self.params['w1'],self.params['b1'],conv_param['stride'],conv_param['pad'])
        self.layers['relu1'] = Relu()
        self.layers['pool1'] = Pooling(pool_h=2,pool_w=2,stride=2)
        self.layers['affinel'] = Affine(self.params['w2'],self.params['b2'])
        self.layers['relu2'] = Relu()
        self.layers['affine2'] = Affine(self.params['w3'],self.params['b3'])
        self.last_layer = SoftmaxWithLoss()

In [10]:
def predict(self,x):
    for layer in self.layers.values():
        x = layer.forward(x)
    return x

def loss(self,x,t):
    y = self.predict(x)
    return self.last_layer.forward(y,t)

In [11]:
def gradient(self,x,t):
    self.loss(x,t)
    
    dout = 1
    dout = self.last_layer.backward(dout)
    
    layers = list(self.layers.values())
    layers.reverse()
    for layer in layers:
        dout = layer.backward(dout)
        
        grads = {}
        grads['w1'] = self.layers['conv1'].dw
        grads['b1'] = self.layers['conv1'].db
        grads['w2'] = self.layers['affine1'].dw
        grads['b2'] = self.layers['affine1'].db
        grads['w3'] = self.layers['affine2'].dw
        grads['b3'] = self.layers['affine2'].db
        
        return grads